## $\color{Green}{\text{GMD Table 4: GMSL from Workflows}}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{Workflow data from Bob's run on Amarel}}$

In [2]:
# Load FACTS workflow data set names as a dataframe.
df_WF = pd.read_fwf( '001_IP_names_Modules_Data/mod-submod-data_WF_global_Bob_amarel_V2.txt' )
df_WF

,Component,Workflow,Data_File
0,total,wf1e,total.workflow.wf1e.global.nc
1,total,wf1f,total.workflow.wf1f.global.nc
2,total,wf2e,total.workflow.wf2e.global.nc
3,total,wf2f,total.workflow.wf2f.global.nc


#### $\color{Red}{\text{Use Workflow data from ==> facts-experiments/221206}}$

In [3]:
MODNO       = df_WF.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.  
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for val in MODNO:
    for scenario in SCENARIO: 
        WORKFLOW = df_WF["Workflow"][val];  COMP = df_WF["Component"][val];  
        # Skip if data is absent.
        if df_WF["Data_File"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/221206/coupling.{arg1}/output/'.format(arg1=scenario)
        dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df_WF["Data_File"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind    = np.where(d_nc["years"].values == YEAR)[0][0]
        GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1000  # Convert to meters.
        # Find Percentiles.
        pcntle  = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        TEMP = [COMP,WORKFLOW, scenario] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Component','Workflow','SSP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
#
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( ['Component', 'Workflow', 'SSP'] )[''] ).unstack().swaplevel( 0,1, axis=1 )
df_ptile1


SSP                             ssp126              ssp245              ssp585
                                                                              
Component Workflow                                                            
total     wf1e      0.40 (0.32 , 0.49)  0.50 (0.42 , 0.59)  0.70 (0.60 , 0.80)
          wf1f      0.41 (0.33 , 0.50)  0.50 (0.41 , 0.59)  0.64 (0.57 , 0.75)
          wf2e      0.46 (0.36 , 0.59)  0.57 (0.47 , 0.73)  0.86 (0.71 , 1.07)
          wf2f      0.47 (0.39 , 0.60)  0.58 (0.49 , 0.75)  0.79 (0.70 , 0.96)

#### $\color{Red}{\text{Use Workflow data from ==> facts-experiments/coupling.}}$

In [4]:
MODNO       = df_WF.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.  
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for val in MODNO:
    for scenario in SCENARIO: 
        WORKFLOW = df_WF["Workflow"][val];  COMP = df_WF["Component"][val];  
        # Skip if data is absent.
        if df_WF["Data_File"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        if scenario == 'ssp585':
            dataFOLDER = '/projects/kopp/facts-experiments/coupling.{arg1}.localhost/output/'.format(arg1=scenario)
            dataFILE   = 'coupling.{arg1}.localhost.'.format(arg1=scenario) + df_WF.loc[val].at['Data_File']
        else:
            dataFOLDER = '/projects/kopp/facts-experiments/coupling.{arg1}/output/'.format(arg1=scenario)
            dataFILE   = 'coupling.{arg1}.'.format(arg1=scenario) + df_WF.loc[val].at['Data_File']
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind    = np.where(d_nc["years"].values == YEAR)[0][0]
        GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1000  # Convert to meters.
        # Find Percentiles.
        pcntle  = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        TEMP = [COMP,WORKFLOW, scenario] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Component','Workflow','SSP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
#
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( ['Component', 'Workflow', 'SSP'] )[''] ).unstack().swaplevel( 0,1, axis=1 )
df_ptile1


SSP                             ssp126              ssp245              ssp585
                                                                              
Component Workflow                                                            
total     wf1e      0.41 (0.33 , 0.50)  0.51 (0.43 , 0.61)  0.72 (0.62 , 0.83)
          wf1f      0.41 (0.33 , 0.50)  0.50 (0.41 , 0.59)  0.67 (0.57 , 0.79)
          wf2e      0.55 (0.44 , 0.71)  0.67 (0.54 , 0.84)  0.89 (0.75 , 1.11)
          wf2f      0.48 (0.39 , 0.61)  0.59 (0.49 , 0.73)  0.80 (0.68 , 1.00)